# 引入模块，读取数据
# 构建计算图（构建网络模型）
# 损失函数与优化器
# 开始训练模型
# 对训练的模型预测结果进行评估

# 1.1 导包和设置随机种子

In [2]:
import torchvision
import torchvision.datasets as datasets #为了下一步加载数据
import torch
import torch.nn.functional as F

import numpy as np 
import random #设置随机数

USE_CUDA = torch.cuda.is_available() #GPU可用的标志位
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)  

if USE_CUDA:
    torch.cuda.manual_seed(1)


# 1.2 加载数据和归一化

In [ ]:
train_mnist = datasets.MNIST(root = './data', train = True, download = True) #训练集
test_mnist = datasets.MNIST(root = './data', train = False, download = True) #测试集

train_X, train_Y = train_mnist.data, train_mnist.targets
test_X, test_Y = test_mnist.data, test_mnist.targets

train_X = train_X.float()
test_X = test_X.float()

train_X = train_X / 255.0        #数据归一化
test_X = test_X / 255.0 


# 2. 构建神经网络

In [8]:
dim_in = 28 * 28
dim_hid = 128
dim_out = 10

class TwoLayerNet(torch.nn.Module):
    def __init__(self, dim_in, dim_hid, dim_out):
        super(TwoLayerNet, self).__init__()
        # define the model architecture
        self.fc1 = torch.nn.Linear(dim_in, dim_hid, bias=True)
        self.fc2 = torch.nn.Linear(dim_hid, dim_out, bias=True)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# 提前定义模型
model = TwoLayerNet(dim_in, dim_hid, dim_out)
if USE_CUDA:
    model = model.cuda()

#提前定义loss函数和优化器
loss_fun = torch.nn.NLLLoss(reduction='sum')
eta = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr=eta)



# 3.训练模型

In [9]:

for i in range(1000):
  #Forward pass
  y_pred = model(train_X)
  
  #Loss
  loss = loss_fun(y_pred, train_Y)
  print(i, loss.item())

  optimizer.zero_grad()
  # Backward pass
  loss.backward()
  
  # update model parameters
  optimizer.step()

# 4.GPU加速训练

In [ ]:
if USE_CUDA:
  train_X = train_X.cuda()
  train_Y = train_Y.cuda()
  test_X = test_X.cuda()
  test_Y = test_Y.cuda()

if USE_CUDA:
    model = model.cuda()

for i in range(1000):
  #Forward pass
  y_pred = model(train_X)
  
  #Loss
  loss = loss_fun(y_pred, train_Y)
  print(i, loss.item())

  optimizer.zero_grad()
  # Backward pass
  loss.backward()
  
  # update model parameters
  optimizer.step()    


In [9]:
# 5.模型评估

In [ ]:
y_pred = model(test_X)
pred = y_pred.argmax(dim=1, keepdim=True) # get the index of the max log-probability
correct = pred.eq(test_Y.view_as(pred)).sum().item() 
acc = correct / test_X.shape[0]
print('准确率为{}%'.format(acc * 100))
